# Ollama Level 101: A Beginner's Complete Guide

Welcome to this comprehensive introduction to Ollama!

## What You'll Learn
1. What is Ollama?
2. Installing and setting up
3. Using the API
4. Python integration
5. Working with real data (ollama_sample_data.csv)
6. Building a Q&A system

## 1. What is Ollama?

**Ollama** runs LLMs locally on your machine.

| Feature | Benefit |
|---------|--------|
| Privacy | Data stays local |
| Cost | Free after setup |
| Offline | Works without internet |
| Customizable | Create specialized models |

In [ ]:
# Check Ollama installation
!which ollama || echo "Install from: https://ollama.com/download"
!ollama --version

In [ ]:
# Install Python packages
!pip install ollama pandas numpy requests

In [ ]:
import ollama
import pandas as pd
import requests
import json

# Check available models
try:
    models = ollama.list()
    print("Available models:")
    for m in models.get('models', []):
        print(f"  - {m['name']}")
except:
    print("Start Ollama server: ollama serve")

## 2. Basic Generation

In [ ]:
# Simple generation
def generate(model, prompt):
    try:
        response = ollama.generate(model=model, prompt=prompt)
        return response['response']
    except Exception as e:
        return f"Error: {e}"

# Example (uncomment if you have a model)
# print(generate('llama3.2:1b', 'What is Python in one sentence?'))

In [ ]:
# Chat conversation
def chat(model, messages):
    try:
        response = ollama.chat(model=model, messages=messages)
        return response['message']['content']
    except Exception as e:
        return f"Error: {e}"

# Example
messages = [
    {'role': 'system', 'content': 'You are helpful.'},
    {'role': 'user', 'content': 'Hello!'}
]
# print(chat('llama3.2:1b', messages))

## 3. Streaming Responses

In [ ]:
def stream_chat(model, prompt):
    try:
        stream = ollama.chat(
            model=model,
            messages=[{'role': 'user', 'content': prompt}],
            stream=True
        )
        for chunk in stream:
            print(chunk['message']['content'], end='', flush=True)
        print()
    except Exception as e:
        print(f"Error: {e}")

# Example
# stream_chat('llama3.2:1b', 'Tell me a short joke.')

## 4. Custom Modelfiles

In [ ]:
# Example Modelfile
modelfile = """FROM llama3.2:1b
SYSTEM You are a Python expert. Be concise.
PARAMETER temperature 0.7
"""

# Save it
with open('Modelfile.python', 'w') as f:
    f.write(modelfile)

print("Modelfile created!")
print("Create model: ollama create python-expert -f Modelfile.python")

## 5. Working with Real Data: ollama_sample_data.csv

Let's process our 100,000 record text dataset!

In [ ]:
# Load dataset
df = pd.read_csv('ollama_sample_data.csv')
print(f"Shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
print(f"\nTopic distribution:\n{df['topic'].value_counts()}")
df.head()

In [ ]:
# Sample questions by topic
print("Sample Questions:\n")
for topic in df['topic'].unique()[:4]:
    sample = df[df['topic'] == topic].iloc[0]
    print(f"[{topic.upper()}] {sample['question']}")
    print(f"Context: {sample['context'][:80]}...\n")

In [ ]:
# Question answering with context
def answer_with_context(model, question, context):
    prompt = f"""Context: {context}

Question: {question}

Answer concisely:"""
    try:
        response = ollama.generate(
            model=model,
            prompt=prompt,
            options={'num_predict': 100, 'temperature': 0.7}
        )
        return response['response']
    except Exception as e:
        return f"Error: {e}"

# Example (uncomment to run)
# sample = df.iloc[0]
# answer = answer_with_context('llama3.2:1b', sample['question'], sample['context'])
# print(f"Q: {sample['question']}")
# print(f"A: {answer}")

In [ ]:
# Batch question classification
def classify_complexity(model, question):
    prompt = f"""Classify as 'simple', 'intermediate', or 'advanced'. One word only.

Question: {question}

Complexity:"""
    try:
        response = ollama.generate(
            model=model, prompt=prompt,
            options={'num_predict': 5, 'temperature': 0.1}
        )
        return response['response'].strip().lower()
    except:
        return 'error'

# Example batch processing
# for i, row in df.head(3).iterrows():
#     pred = classify_complexity('llama3.2:1b', row['question'])
#     print(f"Actual: {row['complexity']}, Pred: {pred}")

## 6. Building a Q&A System

In [ ]:
class SimpleQA:
    def __init__(self, df, model='llama3.2:1b'):
        self.df = df
        self.model = model
    
    def search(self, query, topic=None, n=5):
        """Find relevant questions using keyword matching."""
        subset = self.df if topic is None else self.df[self.df['topic'] == topic]
        query_words = set(query.lower().split())
        
        scores = []
        for idx, row in subset.iterrows():
            q_words = set(row['question'].lower().split())
            overlap = len(query_words & q_words)
            scores.append((idx, overlap, row))
        
        scores.sort(key=lambda x: x[1], reverse=True)
        return scores[:n]
    
    def answer(self, query, topic=None):
        """Find and answer using context."""
        results = self.search(query, topic)
        if not results:
            return None
        
        best = results[0][2]
        return {
            'question': best['question'],
            'topic': best['topic'],
            'context': best['context'],
            'complexity': best['complexity']
        }

# Create Q&A system
qa = SimpleQA(df)

# Test search
result = qa.answer('machine learning')
print(f"Found: {result['question']}")
print(f"Topic: {result['topic']}")

In [ ]:
# Interactive chatbot
class ChatBot:
    def __init__(self, model='llama3.2:1b', system=None):
        self.model = model
        self.messages = []
        if system:
            self.messages.append({'role': 'system', 'content': system})
    
    def chat(self, user_input):
        self.messages.append({'role': 'user', 'content': user_input})
        try:
            response = ollama.chat(model=self.model, messages=self.messages)
            reply = response['message']['content']
            self.messages.append({'role': 'assistant', 'content': reply})
            return reply
        except Exception as e:
            return f"Error: {e}"
    
    def reset(self):
        system = self.messages[0] if self.messages and self.messages[0]['role'] == 'system' else None
        self.messages = [system] if system else []

# Example
# bot = ChatBot('llama3.2:1b', 'You are helpful and concise.')
# print(bot.chat('Hello!'))

## 7. Embeddings (RAG Basics)

In [ ]:
import numpy as np

def get_embedding(text, model='nomic-embed-text'):
    try:
        response = ollama.embeddings(model=model, prompt=text)
        return response['embedding']
    except:
        return None

def cosine_sim(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# Example semantic search
# emb1 = get_embedding('machine learning')
# emb2 = get_embedding('artificial intelligence')
# print(f"Similarity: {cosine_sim(emb1, emb2):.4f}")

## Summary

### What We Covered
- Installing and configuring Ollama
- Generation and chat APIs
- Custom Modelfiles
- Processing text data (100,000 records)
- Building Q&A systems
- Embeddings for semantic search

### Useful Commands
```bash
ollama list          # List models
ollama pull <model>  # Download
ollama run <model>   # Interactive chat
ollama serve         # Start API server
```

### Resources
- [Ollama Website](https://ollama.com/)
- [Ollama Library](https://ollama.com/library)

In [ ]:
# Cleanup
import os
if os.path.exists('Modelfile.python'):
    os.remove('Modelfile.python')